# Super-Resolution for Remote Sensing Imagery

This notebook demonstrates how to enhance the spatial resolution of remote sensing imagery (e.g., Sentinel-2) using a deep learning-based Super-Resolution Convolutional Neural Network (SRCNN) with `torch` in Python. Super-resolution improves image detail, enabling applications like detailed land cover mapping or urban analysis.

## Prerequisites
- Install required libraries: `torch`, `rasterio`, `geopandas`, `numpy`, `matplotlib`, `scikit-image` (listed in `requirements.txt`).
- A preprocessed multi-band Sentinel-2 GeoTIFF (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining the area of interest (AOI) (e.g., `aoi.geojson`).
- Replace file paths with your own data.
- GPU recommended for faster training.

## Learning Objectives
- Load and preprocess Sentinel-2 imagery for super-resolution.
- Train an SRCNN model to enhance image resolution.
- Evaluate the super-resolved imagery using metrics like PSNR and SSIM.
- Visualize and save the enhanced imagery.

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
from rasterio.mask import mask
import os

## Step 1: Load Sentinel-2 Data and AOI

Load a preprocessed Sentinel-2 GeoTIFF and crop it to the AOI.

In [ ]:
# Define file paths
raster_path = 'remote_sensing_data/sentinel_rgb.tif'  # Replace with your Sentinel-2 GeoTIFF
aoi_path = 'aoi.geojson'                             # Replace with your AOI file

# Load raster
with rasterio.open(raster_path) as src:
    raster_data, transform = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)
    raster_profile = src.profile
    raster_crs = src.crs
raster_profile.update({
    'height': raster_data.shape[1],
    'width': raster_data.shape[2],
    'transform': transform,
    'nodata': np.nan
})

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != raster_crs:
    aoi_gdf = aoi_gdf.to_crs(raster_crs)

# Print basic information
print(f'Raster shape: {raster_data.shape}')
print(f'Raster CRS: {raster_crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Prepare Dataset for SRCNN

Create low-resolution (LR) and high-resolution (HR) image pairs for training.

In [ ]:
# Define custom dataset class
class SuperResolutionDataset(Dataset):
    def __init__(self, raster_data, scale_factor=2, patch_size=64):
        self.raster_data = raster_data
        self.scale_factor = scale_factor
        self.patch_size = patch_size
        self.hr_patches = []
        self.lr_patches = []

        # Extract HR and LR patches
        height, width = raster_data.shape[1], raster_data.shape[2]
        for i in range(0, height - patch_size + 1, patch_size//2):
            for j in range(0, width - patch_size + 1, patch_size//2):
                hr_patch = raster_data[:, i:i+patch_size, j:j+patch_size]
                if not np.any(np.isnan(hr_patch)):
                    lr_patch = resize(hr_patch.transpose(1, 2, 0), (patch_size//scale_factor, patch_size//scale_factor), anti_aliasing=True)
                    lr_patch = resize(lr_patch, (patch_size, patch_size), anti_aliasing=False).transpose(2, 0, 1)
                    self.hr_patches.append(hr_patch)
                    self.lr_patches.append(lr_patch)

    def __len__(self):
        return len(self.hr_patches)

    def __getitem__(self, idx):
        hr_patch = self.hr_patches[idx].astype(np.float32)
        lr_patch = self.lr_patches[idx].astype(np.float32)
        return torch.from_numpy(lr_patch), torch.from_numpy(hr_patch)

# Normalize raster data
norm_raster_data = raster_data / np.nanpercentile(raster_data, 98, axis=(1, 2), keepdims=True)
norm_raster_data = np.clip(norm_raster_data, 0, 1)

# Create dataset
dataset = SuperResolutionDataset(norm_raster_data, scale_factor=2, patch_size=64)

# Split dataset into train and validation sets
train_idx, val_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')

## Step 3: Define and Train SRCNN Model

Set up a Super-Resolution Convolutional Neural Network (SRCNN) for enhancing image resolution.

In [ ]:
# Define SRCNN model
class SRCNN(nn.Module):
    def __init__(self, in_channels):
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv2d(in_channels, 64, kernel_size=9, padding=4)
        self.layer2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.layer3 = nn.Conv2d(32, in_channels, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Initialize model
model = SRCNN(in_channels=raster_data.shape[0]).to(device='cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
train_losses, val_losses = [], []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for lr_patches, hr_patches in train_loader:
        lr_patches, hr_patches = lr_patches.to(device), hr_patches.to(device)
        optimizer.zero_grad()
        outputs = model(lr_patches)
        loss = criterion(outputs, hr_patches)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_loader))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for lr_patches, hr_patches in val_loader:
            lr_patches, hr_patches = lr_patches.to(device), hr_patches.to(device)
            outputs = model(lr_patches)
            loss = criterion(outputs, hr_patches)
            val_loss += loss.item()
    val_losses.append(val_loss / len(val_loader))

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

# Save trained model
torch.save(model.state_dict(), 'srcnn_super_resolution.pth')
print('Trained model saved to: srcnn_super_resolution.pth')

## Step 4: Generate Super-Resolved Imagery

Apply the trained SRCNN model to enhance the resolution of the entire raster.

In [ ]:
# Create low-resolution input
scale_factor = 2
lr_data = resize(norm_raster_data.transpose(1, 2, 0), (norm_raster_data.shape[1]//scale_factor, norm_raster_data.shape[2]//scale_factor), anti_aliasing=True)
lr_data = resize(lr_data, (norm_raster_data.shape[1], norm_raster_data.shape[2]), anti_aliasing=False).transpose(2, 0, 1)

# Predict super-resolved imagery
patch_size = 64
height, width = norm_raster_data.shape[1], norm_raster_data.shape[2]
sr_data = np.zeros_like(norm_raster_data, dtype=np.float32)

model.eval()
with torch.no_grad():
    for i in range(0, height - patch_size + 1, patch_size//2):
        for j in range(0, width - patch_size + 1, patch_size//2):
            patch = lr_data[:, i:i+patch_size, j:j+patch_size]
            if not np.any(np.isnan(patch)):
                patch_tensor = torch.from_numpy(patch.astype(np.float32)).unsqueeze(0).to(device)
                sr_patch = model(patch_tensor).cpu().numpy()[0]
                sr_data[:, i:i+patch_size, j:j+patch_size] = sr_patch

# Denormalize super-resolved data
sr_data = sr_data * np.nanpercentile(raster_data, 98, axis=(1, 2), keepdims=True)

# Save super-resolved imagery
sr_profile = raster_profile.copy()
sr_output_path = 'remote_sensing_data/super_resolved.tif'
with rasterio.open(sr_output_path, 'w', **sr_profile) as dst:
    dst.write(sr_data)

print(f'Super-resolved imagery saved to: {sr_output_path}')

## Step 5: Evaluate and Visualize Results

Compare the super-resolved imagery with the original and compute quality metrics.

In [ ]:
# Compute PSNR and SSIM for a sample patch
sample_patch = norm_raster_data[:, :patch_size, :patch_size]
sr_patch = sr_data[:, :patch_size, :patch_size]
psnr_value = psnr(sample_patch.transpose(1, 2, 0), sr_patch.transpose(1, 2, 0), data_range=1.0)
ssim_value = ssim(sample_patch.transpose(1, 2, 0), sr_patch.transpose(1, 2, 0), multichannel=True, data_range=1.0)

print(f'PSNR: {psnr_value:.2f} dB')
print(f'SSIM: {ssim_value:.4f}')

# Visualize original, low-resolution, and super-resolved RGB
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.imshow(norm_raster_data[:3].transpose(1, 2, 0))
ax1.set_title('Original RGB')
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')
ax2.imshow(lr_data[:3].transpose(1, 2, 0))
ax2.set_title('Low-Resolution RGB')
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')
ax3.imshow(sr_data[:3].transpose(1, 2, 0))
ax3.set_title('Super-Resolved RGB')
ax3.set_xlabel('Column')
ax3.set_ylabel('Row')
plt.tight_layout()
plt.show()

## Step 6: Visualize with AOI Overlay

Overlay the AOI on the super-resolved imagery for context.

In [ ]:
# Visualize super-resolved imagery with AOI
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(sr_data[:3].transpose(1, 2, 0))
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.title('Super-Resolved Imagery with AOI Overlay')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `sentinel_rgb.tif` with your own Sentinel-2 GeoTIFF (e.g., from `21_download_data.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Experiment with other super-resolution models (e.g., ESRGAN) or different scale factors.
- Use the super-resolved imagery in downstream tasks like classification (see `12_classification_rf_svm.ipynb` or `27_transfer_learning.ipynb`) or segmentation (see `15_unet_segmentation.ipynb`).
- Visualize results with `23_kepler_gl_demo.ipynb` or `26_time_series_animation.ipynb`.

## Notes
- Ensure input imagery is preprocessed to remove clouds (see `28_cloud_detection_deep_learning.ipynb`).
- Adjust `patch_size` and `scale_factor` based on your dataset and computational resources.
- Super-resolution performance depends on training data quality; consider using high-resolution reference imagery if available.
- See `docs/installation.md` for troubleshooting library installation.